<a target="_blank" href="https://colab.research.google.com/github/alexwolson/postdocbootcamp2023/blob/main/lab_1_2_hyperparameters.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# UofT DSI-CARTE Postdoc Bootcamp
#### Monday June 19, 2023
#### Hyperparameter Tuning - Lab 2, Day 1
#### Teaching team: Alex Olson, Nakul Upadhya, Shehnaz Islam
##### Lab author: Nakul Upadhya (referencing work by Kyle E. C. Booth and Jake Mosseri)

Machine Learning involves solving an optimization problem to find the optimal parameters of a function that best represents the data. However, some parameters cannot be determined by this learning procedure as they define the structure of the model and the optimization process. We call these values hyperparameters, and tuning them is critical to the model development process.

In this lab, we will cover the basics of hyperparameter tuning using Grid Search, Cross Validation, and Bayesian Optimization. Let's get started!

In [ ]:
!pip install torch torchvision torchaudio
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install optuna

import numpy as np
import pandas as pd

## Data Processing

In today's lab, we're going to work with the Titanic dataset. To get it ready for our learning tasks, we need to:
1. Drop unimportant columns
2. Fix the data types of some columns
3. Split into Train and Test sets
4. Impute Missing Values
5. Scale the data

We're going to do this all in the next cells:

In [ ]:
from sklearn.datasets import fetch_openml # import function to retrieve relevant datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Read in the data
data = fetch_openml("titanic", version=1, as_frame=True, parser='auto').frame

# Drop unimportant columns
data = data.drop(['name', 'ticket', 'cabin', 'embarked', 'boat', 'body', 'home.dest'], axis=1) # remove unimportant columns

# Fix Data types
le = LabelEncoder()
data['sex'] = pd.to_numeric(le.fit_transform(data['sex']))
data['survived']= pd.to_numeric(data['survived'])

target_data = data['survived']
feature_data = data.drop('survived', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size=0.25, random_state=0)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(f"There are {X_train.shape[0]} training data points and {X_test.shape[0]} testing points")

There are 981 training data points and 328 testing points


## Grid Search

Now that we have our data set up, we can begin to talk about hyperparameter tuning. The first method we will use is grid search. This involves creating a grid of possible hyperparameter values and exhaustively evaluating the model's performance for each combination. This is commonly combined with K-Fold cross validation to get an accurate representation of the model performance. The Grid Search CV process looks like:

1. Define the hyperparameter grid: Determine the hyperparameters you want to tune and specify the possible values for each hyperparameter. This forms a grid-like structure where each point in the grid represents a unique combination of hyperparameters.


3. Train and evaluate the model across folds: For each combination of hyperparameters, run K-Fold Cross validation with your model and evaluate the average error.

4. Select the best hyperparameters: Choose the set of hyperparameters that yield the best performance (lowest error).


Lets see the impact of hyperparameter tuning with a simple Random Forest Classifier below.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from time import time

clf = RandomForestClassifier() # First we define our model without passing in parameters
hyperparameter_search = { # Then we decide the possible parameter combinations
    'max_depth': [3, 4, 5, 6, 7], # Max Depth of each individual tree
    'n_estimators': [50,100,150,200], # Number of trees generated
    'min_samples_leaf': [1, 2, 4, 8], # Minimum number of samples found in a leaf
    'min_samples_split': [2,4,8,16,32] # Minimum samples required for a split
}

evaluation_metric = make_scorer(accuracy_score, # GridSearchCV requires us to wrap our metric function in a "scorer"
                                greater_is_better = True)
t1 = time()
grid_search_cv = GridSearchCV(estimator = clf,
                              param_grid = hyperparameter_search,
                              scoring = evaluation_metric,
                              cv = 5) # Set up search algorithm

grid_search_cv.fit(X_train, y_train) # Run the search. NOTE: This may take a while
t2 = time()
print(f"Time Taken: {t2-t1} seconds")

print("Best Parameters: ", grid_search_cv.best_params_) # Print the parameters
print ("Best CV Accuracy: ", grid_search_cv.best_score_ * 100, "%")

clf = grid_search_cv.best_estimator_ # Get the best model from the GridSearch
accuracy = accuracy_score(y_test, clf.predict(imp.transform(X_test)))
print ("Testing Accuracy: ", accuracy * 100, "%") # Print the testing accuracy of the best model

Time Taken: 519.3808827400208 seconds
Best Parameters:  {'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 150}
Best CV Accuracy:  81.85330985185952 %
Testing Accuracy:  81.09756097560977 %


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


## Better Parameter Searching
While Grid Search works for many cases, one downside of this procedure is its computational complexity. For each new hyperparameter, the number of models trained increases exponentially. In the example above, there were 1024 combinations ($4^5$), meaning we needed to train a model over five thousand times ($1024 \times 5$ folds). While this can work for a shallow model like a Random Forest or Linear/Logistic regression, the runtime can become prohibitively large for deep models such as Neural Networks or if we have a large search space

### Introduction to Bayesian Optimization
Another hyperparameter approach, Bayesian Optimization, can help alleviate some of these runtime concerns. Bayesian optimization is a sequential model-based optimization technique used to find the optimal configuration of hyperparameters for a given machine learning model. It combines concepts from Bayesian inference and optimization to identify promising regions for evaluation by using a surrogate model to estimate the accuracy of our primary model with respect to its hyperparameters.

Bayesian optimization is advantageous as it offers a more efficient exploration of the hyperparameter space than exhaustive search methods like grid search. By intelligently selecting new configurations based on the surrogate model's predictions, it converges to the optimal hyperparameters more quickly and with fewer evaluations.

Lets try using Bayesian Optimization to search for parameters of our Random Forest. We will code this using Optuna, a hyperparameter optimization package that implements this paradigm:



In [ ]:
import optuna
from sklearn.model_selection import cross_validate

# First we need to create a function that specifies our search space and trains
# our model. Optuna will attempt to change the value of the parameters it passes
# into this function to maximize the output.


def optuna_rf_function(trial):

  # Here we define our search space. Instead of needing to pass in specific values
  # we can tell optuna to look for values in a specific range
  hyperparameter_search = {
    'max_depth': trial.suggest_int('max_depth', 3,7),
    'n_estimators': trial.suggest_int('n_estimators', 50, 200),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1,8),
    'min_samples_split': trial.suggest_int('min_samples_split', 2, 32)
  }
  ## Create the model instance
  model = RandomForestClassifier(**hyperparameter_search)
  ## Evaluate and the CV score
  cv_result = cross_validate(model, X_train, y_train,
                             cv = 5, scoring = 'accuracy')
  return cv_result['test_score'].mean()

# start our Optuna study and specify the direction
study = optuna.create_study(direction='maximize')

# Here we pass in our objective function and the number of combinations we want
# optuna to test (100 for now)
study.optimize(optuna_rf_function, n_trials=100,
               show_progress_bar=True, gc_after_trial=True)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
best_params = study.best_params
best_accuracy = study.best_value

print("Best Parameters: ", best_params)
print("Best CV Accuracy: ", best_accuracy * 100, "%")

best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy: ", accuracy * 100, "%")

Best Parameters:  {'max_depth': 7, 'n_estimators': 78, 'min_samples_leaf': 1, 'min_samples_split': 17}
Best CV Accuracy:  81.7528229565938 %
Testing Accuracy:  80.79268292682927 %


**Your Turn**
* How does the CV accuracy of the grid search and Optuna compare? ______________
* How does the testing accuracy compare? ______________
* Which one ran faster and why do you think that is? ______________

### Tuning a Neural Network
One reason to use Bayesian Optimization is to help with the parameter tuning of neural networks. These models often take much longer to train and can be sensitive to hyperparameters such as the dropout, learning rate, weight decay, batch size, and other parameters. Lets see how we can tune these parameyers for a neural network.

#### Creating our network

Lets create a simple neural network seperated by a ReLu non-linearities. We will also wrap our training and prediction logic within this class to simplify the hyperparameter search process.

In this network, our hyperparameters will be:
1. The batch size used for training
2. The learning rate for training
3. The number of training epochs
3. The size of the hidden dimension of the network.



In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
class TitanicMLP(nn.Module):
  """ Simple two layer network """
  def __init__(self,
               input_dim,
               hidden_dim,
               batch_size,
               learning_rate,
               epochs):
      super(TitanicMLP, self).__init__()

      self.input_dim = input_dim
      self.hidden_dim = hidden_dim
      self.batch_size = batch_size
      self.learning_rate = learning_rate
      self.epochs = epochs

      self.forward_pass = nn.Sequential(
          nn.Linear(input_dim, hidden_dim),
          nn.ReLU(),
          nn.Linear(hidden_dim, 1)
      )


  def forward(self, x):
      return self.forward_pass(x)

  def fit(self, X, y):
    self.train()
    # Create tensors from our inputs
    X_tensor = Variable(torch.Tensor(X_train)).float()
    Y_tensor = Variable(torch.Tensor(y_train.values)).float()
    train_dataset = TensorDataset(X_tensor, Y_tensor)

    # set up the DataLoader
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=self.batch_size,
                              shuffle=True)
    # Define our loss function and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)

    for epoch in range(self.epochs):
      for batch_idx, (features, target) in enumerate(train_loader):

        optimizer.zero_grad()
        outputs = self.forward(features)
        loss = criterion(torch.squeeze(outputs), torch.squeeze(target))
        loss.backward()
        optimizer.step()

  def predict(self, X):
    self.eval()
    with torch.no_grad():
      X_tensor = torch.Tensor(X)
      y_pred = torch.sigmoid(self.forward(X_tensor))
      y_pred = torch.round(y_pred).squeeze().numpy()
      return y_pred


Now lets set up Optuna. Due our network class having a `fit` and `predict` function, we can actually re-use a lot of our code from the Random Forest.

**Your Turn**
* Current the Hyperparameter search has some dummy values passed in. Configure the search for `hidden_dim`, `batch_size`, `learning_rate`, and `epochs` based on values you think are reasonable and run the hyperparameter search.
* Look at the scores generated for the various parameter configurations tested by Optuna. Are they similar to each other, or do they drastically impact the model performance?

In [ ]:
from sklearn.model_selection import StratifiedKFold

def optuna_mlp_function(trial):

  HP = {
    'input_dim': trial.suggest_categorical('input_dim', [X_train.shape[1]]), # Don't change this
    'hidden_dim': trial.suggest_int('hidden_dim', 1, 1e6), # Change this
    'batch_size': trial.suggest_int('batch_size', 1, 1e6), # Change this
    'learning_rate': trial.suggest_float('learning_rate', 1, 1e6), # Change this
    'epochs': trial.suggest_int('epochs', 1,1e6) # Change this
  }
  ## Create the model instance
  model = TitanicMLP(**HP)
  scores = []
  kfold = StratifiedKFold(n_splits = 5)
  for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    X_fold = X_train[train_index]
    y_fold = y_train.values[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train.values[test_index]
    model.fit(X_fold, y_fold)
    y_pred = model.predict(X_test_fold)
    scores.append(accuracy_score(y_test_fold, y_pred))
  return np.mean(scores)
# start our Optuna study and specify the direction
mlp_study = optuna.create_study(direction='maximize')

# Here we pass in our objective function and the number of combinations we want
# optuna to test (100 for now)
mlp_study.optimize(optuna_mlp_function, n_trials=100,
                    show_progress_bar=True, gc_after_trial=True)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
best_params = mlp_study.best_params
best_accuracy = mlp_study.best_value
print("Best Parameters: ", mlp_study.best_params)
print("Best CV Accuracy: ", mlp_study.best_value * 100, "%")


best_model = TitanicMLP(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy: ", accuracy * 100, "%")

Best Parameters:  {'input_dim': 6, 'hidden_dim': 58, 'batch_size': 131, 'learning_rate': 0.0009881690355287936, 'epochs': 232}
Best CV Accuracy:  79.50896094478401 %
Testing Accuracy:  75.3048780487805 %


**Your Turn**
* Apart from the values we searched for above, what other parameters do you think can be tuned in a Neural Network?
